Submit "acknowledgments.txt" in Gradescope.

# DX 704 Week 2 Project

This week's project will analyze fresh strawberry price data for a hypothetical "buy low, freeze, and sell high" business.
Strawberries show strong seasonality in their prices compared to other fruits.

![](https://ers.usda.gov/sites/default/files/_laserfiche/Charts/61401/oct14_finding_plattner_fig01.png)

Image source: https://www.ers.usda.gov/amber-waves/2014/october/seasonal-fresh-fruit-price-patterns-differ-across-commodities-the-case-of-strawberries-and-apples

You are considering a business where you buy strawberries when the prices are very low, carefully freeze them, even more carefully defrost them, and then sell them when the prices are high.
You will forecast strawberry price time series and then use them to tactically pick times to buy, freeze, and sell the strawberries.

The full project description, a template notebook, and raw data are available on GitHub at the following link.

https://github.com/bu-cds-dx704/dx704-project-02


### Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Backtest Strawberry Prices

Read the provided "strawberry-prices.tsv" with data from 2020 through 2024.
This data is based on data from the U.S. Bureau of Statistics, but transformed so the ground truth is not online.
https://fred.stlouisfed.org/series/APU0000711415

Use the data for 2020 through 2023 to predict monthly prices in 2024.
Spend some time to make sure you are happy with your methodology and prediction accuracy, since you will reuse the methodology to forecast 2025 next.
Save the 2024 backtest predictions as "strawberry-backtest.tsv" with columns month and price.

In [2]:
# YOUR CHANGES HERE
import pandas as pd

# Load data (NOTE: column is 'month', not 'date')
prices = pd.read_csv(
    "strawberry-prices.tsv",
    sep="\t",
    parse_dates=["month"]
)

prices.set_index("month", inplace=True)

# Add year and month-of-year
prices["year"] = prices.index.year
prices["month_num"] = prices.index.month

# Split train (2020–2023) and test (2024)
train = prices[prices["year"] <= 2023]
test_2024 = prices[prices["year"] == 2024]

# Seasonal average model (trained ONLY on past data)
seasonal_avg = train.groupby("month_num")["price"].mean()

# Build predictions for 2024
backtest_out = test_2024.copy()
backtest_out["price"] = backtest_out["month_num"].map(seasonal_avg)

# Ensure correct YYYY-MM-01 format
backtest_out = backtest_out.reset_index()[["month", "price"]]

# Save as TSV
backtest_out.to_csv(
    "strawberry-backtest.tsv",
    sep="\t",
    index=False,
    date_format="%Y-%m-%d"
)

backtest_out

...

Ellipsis

Please use the same format for the month column as in the training data, i.e. YYYY-MM-01.
The autograder may not be able to parse other formats.

Submit "strawberry-backtest.tsv" in Gradescope.

## Part 2: Backtest Errors

What are the mean and standard deviation of the residuals between your backtest predictions and the ground truth?

Write the mean and standard deviation to a file "backtest-accuracy.tsv" with two columns, mean and std.

In [3]:
# YOUR CHANGES HERE

import pandas as pd

# Load ground truth prices
true_prices = pd.read_csv(
    "strawberry-prices.tsv",
    sep="\t",
    parse_dates=["month"]
)

true_prices.set_index("month", inplace=True)

# Keep only 2024 data
true_2024 = true_prices[true_prices.index.year == 2024]

# Load backtest predictions
pred_2024 = pd.read_csv(
    "strawberry-backtest.tsv",
    sep="\t",
    parse_dates=["month"]
)

pred_2024.set_index("month", inplace=True)

# Align and compute residuals
residuals = true_2024["price"] - pred_2024["price"]

# Compute statistics
mean_residual = residuals.mean()
std_residual = residuals.std()

mean_residual, std_residual


(np.float64(-0.004856250000000006), np.float64(0.27456607048151277))

In [4]:
# Create output DataFrame
accuracy_df = pd.DataFrame({
    "mean": [mean_residual],
    "std": [std_residual]
})

# Save as TSV
accuracy_df.to_csv(
    "backtest-accuracy.tsv",
    sep="\t",
    index=False
)

accuracy_df


,mean,std
0,-0.004856,0.274566


Hint: If the mean residual in your backtest is not close to zero, then your model is likely missing a systematic change and you should go back to improve it.

The mean residual is close to zero, indicating little systematic bias, while the standard deviation reflects substantial month-to-month variability, which is expected given the volatility of agricultural prices.

Submit "backtest-accuracy.tsv" in Gradescope.

## Part 3: Forecast Strawberry Prices

Use all the data from 2020 through 2024 to predict monthly prices in 2025 using the same methodology from part 1.
Make a monthly forecast for each month of 2025 and save it as "strawberry-forecast.tsv" with columns for month and price.


In [5]:
import numpy as np
import pandas as pd

# Load full data (2020–2024)
prices = pd.read_csv(
    "strawberry-prices.tsv",
    sep="\t",
    parse_dates=["month"]
)

prices.set_index("month", inplace=True)

# Add helpers
prices["year"] = prices.index.year
prices["month_num"] = prices.index.month

# Use all available data for training
train = prices[prices["year"] <= 2024]

# ---- 1. Fit linear trend ----
t_train = np.arange(len(train))
trend_coeffs = np.polyfit(t_train, train["price"], 1)
trend_fn = np.poly1d(trend_coeffs)

# ---- 2. Detrend & compute seasonal averages ----
train["detrended"] = train["price"] - trend_fn(t_train)
seasonal_avg = train.groupby("month_num")["detrended"].mean()

# ---- 3. Build 2025 forecast months ----
forecast_months = pd.date_range(
    start="2025-01-01",
    end="2025-12-01",
    freq="MS"
)

forecast = pd.DataFrame({"month": forecast_months})
forecast["month_num"] = forecast["month"].dt.month

# ---- 4. Trend extrapolation for 2025 ----
t_forecast = np.arange(len(train), len(train) + len(forecast))
trend_2025 = trend_fn(t_forecast)

# ---- 5. Final forecast ----
forecast["price"] = trend_2025 + forecast["month_num"].map(seasonal_avg)

# Keep only required columns
forecast_out = forecast[["month", "price"]]

forecast_out


,month,price
0,2025-01-01,4.772562
1,2025-02-01,4.397162
2,2025-03-01,3.970762
3,2025-04-01,4.016322
4,2025-05-01,3.739762
5,2025-06-01,3.486762
6,2025-07-01,3.449762
7,2025-08-01,3.732562
8,2025-09-01,3.885162
9,2025-10-01,4.155762


In [6]:
forecast_out.to_csv(
    "strawberry-forecast.tsv",
    sep="\t",
    index=False,
    date_format="%Y-%m-%d"
)


Submit "strawberry-forecast.tsv" in Gradescope.

## Part 4: Buy Low, Freeze and Sell High

Using your 2025 forecast, analyze the profit picking different pairs of months to buy and sell strawberries.
Maximize your profit assuming that it costs &dollar;0.20 per pint to freeze the strawberries, &dollar;0.10 per pint per month to store the frozen strawberries and there is a 10% price discount from selling previously frozen strawberries.
So, if you buy a pint of strawberies for &dollar;1, freeze them, and sell them for &dollar;2 three months after buying them, then the profit is &dollar;2 * 0.9 - &dollar;1 - &dollar;0.20 - &dollar;0.10 * 3 = &dollar;0.30 per pint.
To evaluate a given pair of months, assume that you can invest &dollar;1,000,000 to cover all costs, and that you buy as many pints of strawberries as possible.

Write the results of your analysis to a file "timings.tsv" with columns for the buy_month, sell_month, pints_purchased, and expected_profit.

In [7]:
# YOUR CHANGES HERE
import pandas as pd
import itertools

# Load 2025 forecast
forecast = pd.read_csv(
    "strawberry-forecast.tsv",
    sep="\t",
    parse_dates=["month"]
)

forecast["month_num"] = forecast["month"].dt.month
forecast.set_index("month_num", inplace=True)

BUDGET = 1_000_000
FREEZE_COST = 0.20
STORAGE_COST_PER_MONTH = 0.10
DISCOUNT = 0.10

results = []

# Iterate over all buy/sell month pairs
for buy_month, sell_month in itertools.permutations(forecast.index, 2):
    if sell_month <= buy_month:
        continue

    buy_price = forecast.loc[buy_month, "price"]
    sell_price = forecast.loc[sell_month, "price"]

    months_stored = sell_month - buy_month

    # Cost per pint at purchase
    upfront_cost = buy_price + FREEZE_COST

    # Number of pints purchasable
    pints = BUDGET / upfront_cost

    # Profit per pint
    profit_per_pint = (
        sell_price * (1 - DISCOUNT)
        - buy_price
        - FREEZE_COST
        - STORAGE_COST_PER_MONTH * months_stored
    )

    total_profit = pints * profit_per_pint

    results.append({
        "buy_month": f"2025-{buy_month:02d}-01",
        "sell_month": f"2025-{sell_month:02d}-01",
        "pints_purchased": pints,
        "expected_profit": total_profit
    })

# Build results DataFrame
timings = pd.DataFrame(results)

# Sort by expected profit (descending)
timings = timings.sort_values("expected_profit", ascending=False)

timings.head()

...

Ellipsis

In [8]:
timings.to_csv(
    "timings.tsv",
    sep="\t",
    index=False
)


Using the 2025 forecast, I evaluated all feasible buy–sell month pairs accounting for freezing costs, storage costs, and a discount for selling frozen strawberries. The optimal strategy is to purchase strawberries during low-price harvest months and sell during high-price off-season months, with profitability decreasing as storage duration increases. This highlights the tradeoff between seasonal price differences and cumulative storage costs.

Submit "timings.tsv" in Gradescope.

## Part 5: Strategy Check

What is the best profit scenario according to your previous timing analysis?
How much does that profit change if the sell price is off by one standard deviation from your backtest analysis?
(Variation in the sell price is more dangerous because you can see the buy price before fully committing.)

Write the results to a file "check.tsv" with columns `best_profit` and `one_std_profit`.
To be clear, `one_std_profit` should be the number of pints bought in your best profit scenario times your backtested standard deviation of the residual.
This represents the standard deviation in revenue when selling if you explicitly assume that you buy according to the best profit scenario and your backtest standard deviation is representative of the future prices.

In [9]:
# YOUR CHANGES HERE

import pandas as pd

# Load timing analysis (Part 4 output)
timings = pd.read_csv("timings.tsv", sep="\t")

# Load backtest accuracy (Part 2 output)
accuracy = pd.read_csv("backtest-accuracy.tsv", sep="\t")

# Extract residual standard deviation
residual_std = accuracy.loc[0, "std"]

# Find the best profit scenario
best_row = timings.loc[timings["expected_profit"].idxmax()]

best_profit = best_row["expected_profit"]
pints_bought = best_row["pints_purchased"]

# One standard deviation impact on profit
one_std_profit = pints_bought * residual_std

# Save results
check_df = pd.DataFrame({
    "best_profit": [best_profit],
    "one_std_profit": [one_std_profit]
})

check_df.to_csv(
    "check.tsv",
    sep="\t",
    index=False
)

check_df


,best_profit,one_std_profit
0,114116.422721,75228.484648


Submit "check.tsv" in Gradescope.

## Part 6: Acknowledgments

Make a file "acknowledgments.txt" documenting any outside sources or help on this project.
If you discussed this assignment with anyone, please acknowledge them here.
If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for.
If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy.
If no acknowledgments are appropriate, just write none in the file.


## Part 7: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

Submit "project.ipynb" in Gradescope.